In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, confusion_matrix

In [3]:
df=pd.read_csv('E:\\NAAMII\\Machine_learning\\dataset\\train_set.csv')

In [4]:
df.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238,CLASS
0,ID_1,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,...,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710,0
1,ID_2,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,...,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548,1
2,ID_3,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,...,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704,1
3,ID_4,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,...,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803,0
4,ID_5,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,...,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608,0


In [5]:
df.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238,CLASS
0,ID_1,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,...,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710,0
1,ID_2,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,...,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548,1
2,ID_3,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,...,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704,1
3,ID_4,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,...,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803,0
4,ID_5,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,...,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608,0


In [6]:
target = df['CLASS']

In [7]:
features = df.drop(['CLASS', 'ID'], axis=1, errors='ignore')


In [8]:
# Check for infinite and extreme values first
print(f"\nData Quality Check:")
inf_count = np.isinf(features.values).sum()
nan_count = np.isnan(features.values).sum()
print(f"Infinite values: {inf_count}")
print(f"NaN values: {nan_count}")


Data Quality Check:
Infinite values: 4
NaN values: 2668


In [9]:
# Replace infinite values with NaN for statistics calculation
features_clean = features.replace([np.inf, -np.inf], np.nan)
features_clean.head()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608


In [10]:
features_clean.dropna(axis=1, thresh=0.9 * len(features_clean))

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_3229,Feature_3230,Feature_3231,Feature_3232,Feature_3233,Feature_3234,Feature_3235,Feature_3236,Feature_3237,Feature_3238
0,18281.541667,18432.0,9409.650391,0.514708,0.011300,0.045369,2.803803,0.356658,1.803803,564.936250,...,382.968383,382.968383,2214.0,1.0,136.625113,0.061710,0.0,28.154838,4.174959,0.061710
1,20010.083333,20100.0,8303.049072,0.417707,0.014959,0.080294,2.338398,0.429532,1.338398,31.291507,...,452.986164,452.986164,2548.5,1.0,232.564022,0.090548,0.0,27.934229,3.931950,0.090548
2,27260.125000,27437.0,12189.649414,0.447160,0.011428,0.046402,2.782842,0.359345,1.782842,11.965643,...,419.781765,419.781765,3400.0,1.0,233.593529,0.068704,0.0,27.904807,4.085035,0.068704
3,41938.125000,42138.0,17866.433594,0.426019,0.009908,0.034878,3.060655,0.326727,2.060655,8.966286,...,439.023968,439.023968,5424.0,1.0,427.429572,0.078803,0.0,27.870588,4.011726,0.078803
4,41274.125000,41439.0,14315.041992,0.346828,0.013596,0.065680,2.478506,0.403469,1.478506,34.898671,...,485.209184,485.209184,5096.0,1.0,726.731554,0.142608,0.0,28.846909,3.571352,0.142608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46787.916667,47002.0,18052.070312,0.385828,0.010883,0.042086,2.874885,0.347840,1.874885,23.499143,...,466.276055,466.276055,6064.0,1.0,585.547823,0.096561,0.0,28.787507,3.894684,0.096561
311,8420.354167,8493.0,4292.039795,0.510004,0.016911,0.101797,2.145061,0.466555,1.145061,25.293867,...,383.044821,383.044821,987.0,1.0,69.155790,0.070369,0.0,29.625473,4.098452,0.070369
312,37262.750000,37407.0,13950.793945,0.374390,0.012759,0.057837,2.585819,0.386725,1.585819,37.474634,...,469.005263,469.005263,4940.0,1.0,505.566802,0.102341,0.0,26.865256,3.815115,0.102341
313,25081.833333,25251.0,11689.275391,0.466045,0.011197,0.044546,2.820962,0.354489,1.820962,18.321132,...,403.597826,403.597826,2944.0,1.0,215.172554,0.073089,0.0,29.180584,4.055504,0.073089


In [11]:
# Impute remaining NaNs with column mean
features_clean.fillna(features_clean.mean(), inplace=True)


In [12]:
# 3. REMOVE ZERO-VARIANCE FEATURES
var_thresh = VarianceThreshold(threshold=0.0)
X_var = var_thresh.fit_transform(features_clean)

In [13]:
# REMOVE HIGHLY CORRELATED FEATURES
def remove_high_correlation_features(X, threshold=0.95):
    corr_matrix = pd.DataFrame(X).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return pd.DataFrame(X).drop(columns=to_drop, axis=1)

X_clean = remove_high_correlation_features(X_var, threshold=0.95)

In [14]:
X_clean

,0,2,3,4,9,10,12,13,14,15,...,3079,3080,3084,3085,3093,3096,3100,3109,3112,3121
0,18281.541667,9409.650391,0.514708,0.011300,564.936250,179.125654,44.140704,18.149861,0.216274,0.088928,...,0.020695,0.005768,0.004403,0.012050,32.099384,0.003989,0.003850,28.957580,0.003820,28.154838
1,20010.083333,8303.049072,0.417707,0.014959,31.291507,122.447882,27.150254,14.165947,0.207880,0.108412,...,0.052191,0.003585,0.002466,0.009676,30.100056,0.003342,0.003069,28.665010,0.003124,27.934229
2,27260.125000,12189.649414,0.447160,0.011428,11.965643,241.904940,47.572298,22.613842,0.161702,0.076866,...,0.022511,0.003818,0.002143,0.016889,30.148227,0.003562,0.002487,28.948552,0.003357,27.904807
3,41938.125000,17866.433594,0.426019,0.009908,8.966286,226.260911,34.835854,29.188142,0.135663,0.113669,...,0.022961,0.004408,0.002557,0.017470,30.898766,0.003814,0.002860,29.198077,0.003292,27.870588
4,41274.125000,14315.041992,0.346828,0.013596,34.898671,208.619270,45.788460,20.180269,0.175847,0.077501,...,0.045295,0.003538,0.001756,0.013465,31.877879,0.003408,0.002936,31.136644,0.002901,28.846909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,46787.916667,18052.070312,0.385828,0.010883,23.499143,278.217541,48.569993,21.849623,0.149795,0.067387,...,0.039386,0.003363,0.002989,0.004861,30.297602,0.003033,0.003033,28.787507,0.003033,28.787507
311,8420.354167,4292.039795,0.510004,0.016911,25.293867,93.618836,18.100903,13.007549,0.181568,0.131103,...,0.028983,0.006227,0.003642,0.023425,32.628572,0.005473,0.004316,30.582970,0.003948,29.625473
312,37262.750000,13950.793945,0.374390,0.012759,37.474634,202.610464,38.141866,28.135131,0.156558,0.115484,...,0.022950,0.003339,0.001783,0.009559,29.512201,0.002889,0.002889,26.865256,0.002889,26.865256
313,25081.833333,11689.275391,0.466045,0.011197,18.321132,204.320826,47.209078,25.232333,0.194147,0.103768,...,0.063719,0.003918,0.003684,0.004855,31.186746,0.003616,0.003616,29.180584,0.003616,29.180584


In [15]:
# 5. SCALING
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_var)  # or X_clean if using correlation pruning

# 6. DIMENSIONALITY REDUCTION WITH PCA (retain 95% variance)
pca = PCA(n_components=0.9, random_state=42)
X_pca = pca.fit_transform(X_scaled)

In [16]:
X_pca.shape

(315, 28)

In [17]:
y=df['CLASS']
y

0      0
1      1
2      1
3      0
4      0
      ..
310    0
311    1
312    0
313    1
314    0
Name: CLASS, Length: 315, dtype: int64

In [27]:


# Helper function to calculate all metrics
def evaluate(y_true, y_pred, y_proba):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Specificity': specificity,
        'AUROC': roc_auc_score(y_true, y_proba)
    }

# Define models
models = {
    "LogisticRegression": LogisticRegression(class_weight='balanced' ,random_state=42,C=0.01,l1_ratio=0.9,solver= 'saga',penalty='elasticnet'),
    "RandomForest": RandomForestClassifier(class_weight='balanced', n_estimators=100, random_state=42,max_depth=None,min_samples_leaf=2, min_samples_split=5),
    "XGBoost": XGBClassifier(scale_pos_weight=191/124, eval_metric='logloss', random_state=42,learing_rate=0.2, n_estimators=200, max_depth=3, subsample=0.8, colsample_bytree=0.8, use_label_encoder=False, verbosity=0),
    "SVM":  SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42,C= 0.001),
    "LightGBM": LGBMClassifier(class_weight='balanced', random_state=42 , learning_rate=0.01, max_depth= 3, n_estimators= 200),
    "NaiveBayes": GaussianNB(var_smoothing= 1e-09)
}

# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {model_name: [] for model_name in models}

# Loop through models and perform cross-validation
for model_name, model in models.items():
    print(f"\nTraining {model_name}")
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_pca, y)):
        X_train, X_val = X_pca[train_idx], X_pca[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)[:, 1]
        
        metrics = evaluate(y_val, y_pred, y_proba)
        results[model_name].append(metrics)
        print(f" Fold {fold+1}: " + ", ".join([f"{k}: {v:.4f}" for k, v in metrics.items()]))

# Average results
print("\n📊 Average Cross-Validation Results:")
for model_name, folds in results.items():
    avg = {k: np.mean([fold[k] for fold in folds]) for k in folds[0]}
    print(f" {model_name}: " + ", ".join([f"{k}: {v:.4f}" for k, v in avg.items()]))



Training LogisticRegression
 Fold 1: Accuracy: 0.5079, F1: 0.4561, Recall: 0.5417, Specificity: 0.4872, AUROC: 0.5780
 Fold 2: Accuracy: 0.5556, F1: 0.5484, Recall: 0.6800, Specificity: 0.4737, AUROC: 0.6421
 Fold 3: Accuracy: 0.6508, F1: 0.6071, Recall: 0.6800, Specificity: 0.6316, AUROC: 0.7179
 Fold 4: Accuracy: 0.6667, F1: 0.6316, Recall: 0.7200, Specificity: 0.6316, AUROC: 0.6558
 Fold 5: Accuracy: 0.6032, F1: 0.5763, Recall: 0.6800, Specificity: 0.5526, AUROC: 0.6632

Training RandomForest
 Fold 1: Accuracy: 0.5397, F1: 0.2927, Recall: 0.2500, Specificity: 0.7179, AUROC: 0.5363
 Fold 2: Accuracy: 0.6032, F1: 0.4898, Recall: 0.4800, Specificity: 0.6842, AUROC: 0.6000
 Fold 3: Accuracy: 0.5873, F1: 0.3500, Recall: 0.2800, Specificity: 0.7895, AUROC: 0.6474
 Fold 4: Accuracy: 0.5714, F1: 0.1818, Recall: 0.1200, Specificity: 0.8684, AUROC: 0.5663
 Fold 5: Accuracy: 0.5714, F1: 0.4255, Recall: 0.4000, Specificity: 0.6842, AUROC: 0.5789

Training XGBoost
 Fold 1: Accuracy: 0.6032, F1:

e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 99, number of negative: 153
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2399
[LightGBM] [Info] Number of data points in the train set: 252, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
e:\NAAMII\Machine_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 Fold 1: Accuracy: 0.5556, F1: 0.3913, Recall: 0.3750, Specificity: 0.6667, AUROC: 0.4936
 Fold 2: Accuracy: 0.5873, F1: 0.5938, Recall: 0.7600, Specificity: 0.4737, AUROC: 0.6726
 Fold 3: Accuracy: 0.6349, F1: 0.3784, Recall: 0.2800, Specificity: 0.8684, AUROC: 0.6884
 Fold 4: Accuracy: 0.5714, F1: 0.3721, Recall: 0.3200, Specificity: 0.7368, AUROC: 0.5221
 Fold 5: Accuracy: 0.5397, F1: 0.4727, Recall: 0.5200, Specificity: 0.5526, AUROC: 0.5947

📊 Average Cross-Validation Results:
 LogisticRegression: Accuracy: 0.5968, F1: 0.5639, Recall: 0.6603, Specificity: 0.5553, AUROC: 0.6514
 RandomForest: Accuracy: 0.5746, F1: 0.3480, Recall: 0.3060, Specificity: 0.7489, AUROC: 0.5858
 XGBoost: Accuracy: 0.5968, F1: 0.4617, Recall: 0.4517, Specificity: 0.6911, AUROC: 0.6042
 SVM: Accuracy: 0.6349, F1: 0.5840, Recall: 0.6447, Specificity: 0.6286, AUROC: 0.6501
 LightGBM: Accuracy: 0.5778, F1: 0.4734, Recall: 0.4843, Specificity: 0.6387, AUROC: 0.5941
 NaiveBayes: Accuracy: 0.5778, F1: 0.4417, Re

In [ ]:
LogisticRegression: Accuracy: 0.5968, F1: 0.5639, Recall: 0.6603, Specificity: 0.5553, AUROC: 0.6514
 RandomForest: Accuracy: 0.5746, F1: 0.3480, Recall: 0.3060, Specificity: 0.7489, AUROC: 0.5858
 XGBoost: Accuracy: 0.5968, F1: 0.4617, Recall: 0.4517, Specificity: 0.6911, AUROC: 0.6042

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Build param grid
param_grid = [
    {
        'penalty': ['l1'],
        'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10],
        'solver': ['liblinear'],
        'class_weight': ['balanced']
    },
    {
        'penalty': ['l2'],
        'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10],
        'solver': ['liblinear', 'lbfgs'],
        'class_weight': ['balanced']
    },
    {
        'penalty': ['elasticnet'],
        'C': [0.01, 0.1, 1, 5],
        'solver': ['saga'],
        'l1_ratio': [0.1, 0.5, 0.7, 0.9],
        'class_weight': ['balanced']
    }
]

logreg = LogisticRegression(max_iter=1000, random_state=42)

# Grid search with cross-validation
log_cv = GridSearchCV(
    logreg,
    param_grid,
    scoring='f1',       # Or 'accuracy' if that's your main target
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit
log_cv.fit(X_pca, y)

# Output best model and params
print("Best parameters:", log_cv.best_params_)
print("Best F1 score:", log_cv.best_score_)


Fitting 5 folds for each of 46 candidates, totalling 230 fits
Best parameters: {'C': 0.01, 'class_weight': 'balanced', 'l1_ratio': 0.9, 'penalty': 'elasticnet', 'solver': 'saga'}
Best F1 score: 0.575707095046355


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_cv = GridSearchCV(rf, rf_grid, scoring='f1', cv=5)
rf_cv.fit(X_pca, y)

print("Best Random Forest params:", rf_cv.best_params_)


In [ ]:
from xgboost import XGBClassifier

xgb_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb = XGBClassifier(scale_pos_weight=191/124, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_cv = GridSearchCV(xgb, xgb_grid, scoring='f1', cv=5)
xgb_cv.fit(X_pca, y)

print("Best XGBoost params:", xgb_cv.best_params_)


In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svm_params = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear'],
    'class_weight': ['balanced']
}

svm = SVC(probability=True, random_state=42)
svm_cv = GridSearchCV(svm, svm_params, scoring='f1', cv=5, n_jobs=-1, verbose=2)
svm_cv.fit(X_pca, y)
print("Best SVM params:", svm_cv.best_params_)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best SVM params: {'C': 0.001, 'class_weight': 'balanced', 'kernel': 'linear'}


In [21]:
from lightgbm import LGBMClassifier

lgbm_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'class_weight': ['balanced']
}

lgbm = LGBMClassifier(random_state=42)
lgbm_cv = GridSearchCV(lgbm, lgbm_params, scoring='f1', cv=5, n_jobs=-1, verbose=2)
lgbm_cv.fit(X_pca, y)
print("Best LGBM params:", lgbm_cv.best_params_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[LightGBM] [Info] Number of positive: 124, number of negative: 191
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000453 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2968
[LightGBM] [Info] Number of data points in the train set: 315, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

In [22]:
from sklearn.naive_bayes import GaussianNB

nb_params = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

nb = GaussianNB()
nb_cv = GridSearchCV(nb, nb_params, scoring='f1', cv=5, n_jobs=-1, verbose=2)
nb_cv.fit(X_pca, y)
print("Best Naive Bayes params:", nb_cv.best_params_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best Naive Bayes params: {'var_smoothing': 1e-09}
